In [6]:
# Import extensions, connect to database and pull data into a dataframe

import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv


# Connect to database 
# Load .env variables
load_dotenv()

# Database connection parameters from .env file
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Create connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

engine = create_engine(connection_string)
sql_query = "SELECT * FROM ecommerce_schema.oct_nov_19"

# Save to dataframe
df = pd.read_sql(sql_query, engine)

# Print few lines to check data in the dataframe
print(df.head())


Empty DataFrame
Columns: [event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session]
Index: []


In [7]:
# Initial analaysis to understand the data

# Get basic info on the dataset
print(df.info())
# # Get summary stats 
# print(df.describe()) 





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   event_time     0 non-null      object
 1   event_type     0 non-null      object
 2   product_id     0 non-null      object
 3   category_id    0 non-null      object
 4   category_code  0 non-null      object
 5   brand          0 non-null      object
 6   price          0 non-null      object
 7   user_id        0 non-null      object
 8   user_session   0 non-null      object
dtypes: object(9)
memory usage: 124.0+ bytes
None
